<img src="GeorgiaTech_Navy.png" width=200/>

# Recipe Automator
A CS7643 Project. 

## Team Members
* Peter Veikos
* Luis Murrugarra
* Esteban Fajardo

## App Instructions
Upload your image and get the best closest recipe from [``allrecipes.com``](https://www.allrecipes.com/) and [``yummly.com``](https://www.yummly.com/).

In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from similarity import *
from recipe_scrapers import scrape_me

In [2]:
image_path = Path("data")

In [3]:
# Read pre-trained model
learn_inf = load_learner('model.pkl')
learn_similar = load_learner('similarity_v2.pkl')

In [4]:
def get_similarity(path, base_img):
    img = PILImage.create(path)
    simg = SiameseImage(img, base_img)
    with learn_similar.no_bar():
        pred, pred_idx, pred_prob = learn_similar.predict(simg)
    return (path, bool(pred), float(pred_prob[pred_idx]))

In [5]:
# Define upload buttom 
btn_upload = widgets.FileUpload(
    accept='.jpg',
    multiple=False
)

input_img_out = widgets.Output()
recipe_img_out = widgets.Output()
lbl_pred = widgets.HTML()
recipe_pred = widgets.HTML()
btn_run = widgets.Button(description='Classify!')

def on_click_classify(change):
    # Get image
    img = PILImage.create(btn_upload.data[-1])
    
    # Delete previous image from the output
    input_img_out.clear_output()
    recipe_img_out.clear_output()
    lbl_pred.value = ""
    recipe_pred.value = ""
    
    with input_img_out: 
        display(widgets.HTML('<h3> Input Image: </h3>'))
        display(img.to_thumb(512,512))
    
    # Do the analytics
    output_html = '<h3> Food Prediction: </h3>'
    output_html += '<div style="border:0.5px dotted red;padding:2%;">'
    
    # Get predicted category
    pred, pred_idx, probs = learn_inf.predict(img)
    output_html += f'<h4> The image is {pred} with probability: {100*probs[pred_idx]:.02f}% </h4>'
    output_html += '</div>'
    lbl_pred.value = output_html
    
    # Find similar recipes
    recipe_html = '<h3> Recipe: </h3>'
    recipe_html += '<div style="border:0.5px dotted blue;padding:2%;">'
    candidates = get_image_files(image_path / pred)
    preds = []
    for cand in progress_bar(candidates):
        preds.append(get_similarity(cand, img))
    top_candidates = list(filter(lambda x:x[1], sorted(preds, key=lambda x:x[2])))
    
    if len(top_candidates) > 0:
        # Get details of closest recipe
        top_path, _, top_prob = top_candidates[-1]
        top_details = pd.read_csv("recipe_table.csv").query(f"image_path=='{'/'.join(top_path.parts[1:])}'").iloc[0].to_dict()
        
        # Scrape recipe details
        scraper = scrape_me(top_details["url"])
        recipe_html += f'<h4> {scraper.title()} </h4>'
        
        # Create a list of attributes
        recipe_html += '<ul>'
        recipe_html += f'<li> <h5> Link: <a href="{top_details["url"]}"> Recipe </a> </h5>'
        
        scrap_details = {"Yields": "yields", "Duration": "total_time", "Calories": "nutrients"}
        for attr, key_scrap in scrap_details.items():
            if attr == "Calories":
                text_out = scraper.nutrients()["calories"]
            elif attr == "Duration":
                text_out = str(getattr(scraper, key_scrap)()) + " minutes"
            else:
                text_out = getattr(scraper, key_scrap)()
            recipe_html += f"<li> <h5> {attr}: {text_out} </h5> </li>"
        
        recipe_img = PILImage.create(top_path)
        with recipe_img_out: 
            display(recipe_img.to_thumb(512,512))
        
        recipe_html += f'<li> <h5> Model Confidence: {100*top_prob:.02f}% </h5> </li>'
        recipe_html += '</ul>'
    else:
        recipe_html += '<h4 style="color:red;"> Cannot find a similar recipe, sorry! </h4>'
    
    recipe_html += '</div>'
    recipe_pred.value = recipe_html

btn_run.on_click(on_click_classify)

In [6]:
# Page Design
VBox([widgets.HTML('<h1> Select your food! </h1>'), 
      widgets.HTML('Upload your image by clicking the upload button and then press the "Classify!" button'),
      btn_upload,
      btn_run,
      input_img_out, 
      lbl_pred,
      recipe_pred,
      recipe_img_out
     ])